In [ ]:
# Directive pour afficher les graphiques dans Jupyter
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
# SeaBorn : librairie de graphiques avancés
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
# Lecture des données d'apprentissage et de test
d = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")

In [ ]:
d.head()

> ## Interprétation des paramètres

- Pregnancies: Number of times pregnant
- Glucose: Plasma glucose concentration a 2 hours in an oral glucose tolerance test
- BloodPressure: Diastolic blood pressure (mm Hg)
- SkinThickness: Triceps skin fold thickness (mm)
- Insulin: 2-Hour serum insulin (mu U/ml)
- BMI: Body mass index (weight in kg/(height in m)^2)
- DiabetesPedigreeFunction: Diabetes pedigree function
- Age: Age (years)
- Outcome: Class variable (0 or 1) 268 of 768 are 1, the others are 0

* ## Quelques visualisations sur le dataset

Visualisation des grossesses

In [ ]:
plt.hist(d.Pregnancies, bins=10)
plt.title("Grossesses")

In [ ]:
plt.hist(d.SkinThickness, bins=40)
plt.title("SkinThickness")

**Commentaire:** On remarque un manque de données important,

Relation grossesses/age:

In [ ]:
sns.jointplot("Pregnancies", "Age", d, kind='kde');

### Données manquantes

In [ ]:
d[d.SkinThickness == 0].count()

In [ ]:
d.SkinThickness = d.SkinThickness.replace(to_replace = 0, value = d.SkinThickness.mean())

In [ ]:
plt.hist(d.SkinThickness, bins=40)
plt.title("SkinThickness")

On fait de même pour l'insuline et la pression du sang

In [ ]:
d.Insulin = d.Insulin.replace(to_replace = 0, value = d.Insulin.mean())

In [ ]:
d.BloodPressure = d.BloodPressure.replace(to_replace = 0, value = d.BloodPressure.mean())

## Déséquilibre des distributions

In [ ]:
sns.distplot(d.Glucose, color='red')

In [ ]:
sns.distplot(d.BloodPressure, color='green')

In [ ]:
sns.distplot(d.BMI, color='blue')

On remarque qu'il n'y a pas de déséquillibres particuliers dans les données.

In [ ]:
sns.kdeplot(d.BMI, color='green')
sns.kdeplot(d.Age, color='red')

## Création des jeux d'apprentissage et de test

In [ ]:
X = d.drop(['Outcome'], axis=1)
y = d.Outcome

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
print(X_train.shape)
print(X_test.shape)

## Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(max_iter = 10000)
lr.fit(X_train,y_train)

In [ ]:
y_lr = lr.predict(X_test)

### Mesurons les performances de la méthode

In [ ]:
# Importation des méthodes de mesure de performances
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score,auc, accuracy_score

Prédictions possibles de Outcome:
* Vrais positifs (VP ou TP) : on prédit "1" et la valeur attendue est "0"
* Vrais négatifs (VN ou TN) : on prédit "0" et la valeur attendue est "0"
* Faux positifs (FP) : on prédit "1" et la valeur attendue est "0"
* Faux négatifs (FN) : on prédit "0" et la valeur attendue est "1"

In [ ]:
print("Score de précision: " + str(accuracy_score(y_test,y_lr)))

In [ ]:
print(confusion_matrix(y_test,y_lr))

In [ ]:
print(classification_report(y_test, y_lr))

In [ ]:
probas = lr.predict_proba(X_test)
print(probas)

On met les probabilités de prédiction de la valeur 1 dans un dataframe, avec les valeurs effectives, pour faciliter la visualisation :

In [ ]:
dfprobas = pd.DataFrame(probas,columns=['proba_0','proba_1'])
dfprobas['y'] = np.array(y_test)

In [ ]:
dfprobas

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(1-dfprobas.proba_0[dfprobas.y==0], bins=50)
sns.distplot(dfprobas.proba_1[dfprobas.y==1], bins=50)

### Courbe ROC :

In [ ]:
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test,probas[:, 1])
roc_auc = auc(false_positive_rate, true_positive_rate)
plt.figure(figsize=(12,12))
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b', label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')        # plus mauvaise courbe
plt.plot([0,0,1],[0,1,1],'g:')     # meilleure courbe
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')

### Ajustement des hyperparamètres (Random Forests)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestClassifier()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)

In [ ]:
print(accuracy_score(y_test, y_rf))

Remarquons que l'accuracy score n'a pas changé avec l'usage de la méthode RF, alors que la précision la précision a legereme varié

In [ ]:
print(classification_report(y_test, y_rf))

In [ ]:
cm = confusion_matrix(y_test, y_rf)
print(cm)

estimation

### GridSearchCV

In [ ]:
from sklearn import model_selection

In [ ]:
param_grid = {
              'n_estimators': [10, 100, 500],
              'min_samples_leaf': [1, 20, 50]
             }
estimator = ensemble.RandomForestClassifier()
rf_gs = model_selection.GridSearchCV(estimator, param_grid)

In [ ]:
rf_gs.fit(X_train, y_train)

In [ ]:
print(rf_gs.best_params_)

* On prend donc le meilleur estimateur

In [ ]:
rf2 = rf_gs.best_estimator_
y_rf2 = rf2.predict(X_test)
print(classification_report(y_test, y_rf2))

Les performances du modèle sont meilleures

### Importance des caractéristiques

In [ ]:
importances = rf2.feature_importances_
indices = np.argsort(importances)

In [ ]:
plt.figure(figsize=(8,5))
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), X_train.columns[indices])
plt.title('Importance des caractéristiques')

### XGBoost

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBClassifier()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
cm = confusion_matrix(y_test, y_xgb)
print(cm)
print(classification_report(y_test, y_xgb))

In [ ]:
def replace_0(df,col) :
    df1 = df.copy()
    n = df.shape[0]
    m = df[col].mean()
    s = df[col].std()
    for i in range(n) :
        if df.loc[i,col]==0 :
            df1.loc[i,col] = np.random.normal(m,s)
    return df1
df = replace_0(d,'Glucose')

In [ ]:
df

la fonction remplace les données manquantes par des nombres aléatoires qui suivent une loi nomrale de paramétres (moyenne m et écart type s).